# PaddlePaddle BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
# !/bin/bash ./utils/setup.sh

In [7]:
!pip install paddlepaddle paddlenlp

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 4.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 64.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 31.3 MB/s eta 0:00:00
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=1931447

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [2]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-PaddleNLP'

role = sagemaker.get_execution_role()

In [3]:
!python prepare.py \
    --mode 'folder' \
    --input_path '../Annotated_Data/Data_Mining' \
    --output_folder './output'

# Prepare data

In [4]:
!python doccano.py \
    --folder_path ./output \
    --task_type ext \
    --save_dir ./data \
    --splits 0.9 0.1 0

[2022-09-21 03:29:43,381] [    INFO] - Converting doccano data...
100%|██████████████████████████████████████| 331/331 [00:00<00:00, 27775.74it/s]
[2022-09-21 03:29:43,394] [    INFO] - Adding negative samples for first stage prompt...
100%|██████████████████████████████████████| 331/331 [00:00<00:00, 45665.24it/s]
[2022-09-21 03:29:43,402] [    INFO] - Adding negative samples for second stage prompt...
100%|██████████████████████████████████████| 331/331 [00:00<00:00, 14063.15it/s]
[2022-09-21 03:29:43,427] [    INFO] - Converting doccano data...
100%|████████████████████████████████████████| 37/37 [00:00<00:00, 49676.46it/s]
[2022-09-21 03:29:43,429] [    INFO] - Adding negative samples for first stage prompt...
100%|████████████████████████████████████████| 37/37 [00:00<00:00, 64743.12it/s]
[2022-09-21 03:29:43,430] [    INFO] - Converting doccano data...
0it [00:00, ?it/s]
[2022-09-21 03:29:43,430] [    INFO] - Adding negative samples for first stage prompt...
0it [00:00, ?it/s]
[2

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [5]:
data_location = sagemaker.Session().upload_data(path = "./data", key_prefix=prefix)
# base_dir = 'file:///home/ec2-user/SageMaker/paddlenlp_sagemaker/data/'
# inputs = {'training': base_dir}
# print(inputs)

In [6]:
data_location

's3://sagemaker-us-east-1-726335585155/sagemaker/DEMO-PaddleNLP'

## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


## SageMaker Training using GPU instance

In [7]:
inputs = {'training': data_location}

print(inputs)

{'training': 's3://sagemaker-us-east-1-726335585155/sagemaker/DEMO-PaddleNLP'}


In [8]:
#upload uie-base-en pretrain

# uie_en_model_s3 = sagemaker.Session().upload_data(path = "../uie-base-en/taskflow/information_extraction/uie-base-en", key_prefix="model_uie_base_en")
uie_en_model_s3 = 's3://sagemaker-us-east-1-726335585155/model_uie_base_en'

In [9]:
from sagemaker.pytorch import PyTorch

hyperparameters = {'train_path': '/opt/ml/input/data/training/train.txt', 
                   'dev_path': '/opt/ml/input/data/training/dev.txt', 
                   'save_dir': '/opt/ml/model', 
                   'learning_rate': 1e-5, 
                   'batch_size': 16, 
                   'max_seq_len':512, 
                   'num_epochs': 20, 
                   'model': 'uie-base',
                   'seed': 1000,
                   'logging_steps': 10,
                   'valid_steps': 1000,
                   'device': 'gpu'}

instance_type = 'ml.p3.8xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

#git_config = {'repo': 'https://github.com/whn09/paddlenlp_sagemaker.git', 'branch': 'main'}

estimator = PyTorch(entry_point='finetune.py',
                    source_dir='./',
                           # git_config=git_config,
                    role=role,
                    hyperparameters=hyperparameters,
                    framework_version='1.9.1',
                    py_version='py38',
                    script_mode=True,
                    instance_count=1,  # 1 or 2 or ...
                    instance_type=instance_type,
                    # Parameters required to enable checkpointing
                    checkpoint_s3_uri=uie_en_model_s3, #使用你自己用来保存/加载模型的s3桶地址, 注意桶需要在us-east-1
                    checkpoint_local_path="/opt/ml/checkpoints")

estimator.fit(inputs)

KeyboardInterrupt: 

In [16]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'xxx'
print(training_job_name)

pytorch-training-2022-09-15-06-58-11-844


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [17]:
!rm -rf model.tar.gz
!rm -rf model_*
!rm -rf inference.*
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

download: s3://sagemaker-us-east-1-726335585155/pytorch-training-2022-09-15-06-58-11-844/output/model.tar.gz to ./model.tar.gz
inference.pdiparams
model_10/
model_10/tokenizer_config.json
model_10/model_config.json
model_10/vocab.txt
model_10/special_tokens_map.json
model_10/model_state.pdparams
inference.pdmodel
model_best/
model_best/tokenizer_config.json
model_best/model_config.json
model_best/vocab.txt
model_best/special_tokens_map.json
model_best/model_state.pdparams
model_40/
model_40/tokenizer_config.json
model_40/model_config.json
model_40/vocab.txt
model_40/special_tokens_map.json
model_40/model_state.pdparams
model_30/
model_30/tokenizer_config.json
model_30/model_config.json
model_30/vocab.txt
model_30/special_tokens_map.json
model_30/model_state.pdparams
model_20/
model_20/tokenizer_config.json
model_20/model_config.json
model_20/vocab.txt
model_20/special_tokens_map.json
model_20/model_state.pdparams
inference.pdiparams.info


In [18]:
!cp inference.* model/
!cp model/code/requirements_gpu.txt model/code/requirements.txt
!cd model && tar -czvf ../model-inference-gpu.tar.gz *

!aws s3 cp model-inference-gpu.tar.gz s3://$bucket/$training_job_name/output/model-inference-gpu.tar.gz

code/
code/requirements.txt
code/infer.py
code/uie_predictor.py
code/infer_cpu.py
code/model.py
code/requirements_cpu.txt
code/requirements_gpu.txt
code/infer_gpu.py
inference.pdiparams
inference.pdiparams.info
inference.pdmodel
upload: ./model-inference-gpu.tar.gz to s3://sagemaker-us-east-1-726335585155/pytorch-training-2022-09-15-06-58-11-844/output/model-inference-gpu.tar.gz


In [19]:
# instance_type = 'local'
# instance_type = 'ml.m5.xlarge'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-inference-gpu.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='infer_gpu.py', framework_version='1.9.0', py_version='py38', model_server_workers=4)  # TODO [For GPU], model_server_workers=6

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

---------------!

In [ ]:
# # endpoint_name = 'pytorch-inference-2022-07-05-07-28-16-183'  # m5.2xlarge
# # endpoint_name = 'pytorch-inference-2022-07-06-04-02-11-091'  # g4dn.xlarge, 6 threads
# endpoint_name = 'pytorch-inference-2022-07-06-06-19-21-855'  # g4dn.xlarge, 4 threads
# predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)

# Invoking the endpoint

In [20]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [21]:
texts = ['"She had long silky-looking white hair']

import time
start = time.time()
outputs = predictor.predict(texts)
end = time.time()
print('outputs: ', outputs)
print('time:', end-start)

# for i in range(1000):
#     start = time.time()
#     outputs = predictor.predict(texts)
#     end = time.time()
#     print('time:', end-start)

outputs:  [{}]
time: 0.27838563919067383


# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()